In [8]:
# YOLOv8
import ultralytics
from ultralytics import YOLO

import numpy as np
import cv2
import torch
import torch.backends.cudnn as cudnn

import supervision as sv

from collections import defaultdict
from tqdm.notebook import tqdm

In [9]:
print('CUDA disponível' if torch.cuda.is_available() else 'CUDA indisponível')
print(f'Ultralytics: {ultralytics.__version__}')
print(f'Torch: {torch.__version__}')
print(f'supervision: {sv.__version__}')

CUDA disponível
Ultralytics: 8.0.106
Torch: 2.1.1+cu121
supervision: 0.16.0


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Carregando o modelo pré-treinado
model = YOLO('yolov8n.pt')
model.fuse()

YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients


cuda


In [11]:
print(model.model.names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [12]:
# Selecionando apenas as classes relevantes pro problema
classes = [2, 7]

In [13]:
# Carrega o vídeo
VIDEO = './dataset/road_video001.mp4'
OUTPUT = 'output_video.mp4'

video_info = sv.VideoInfo.from_video_path(VIDEO)
print(video_info)

# Linha para contagem de carros
START = sv.Point(0, 2*video_info.height//3)
END = sv.Point(video_info.width, 2*video_info.height//3)

carros = 0

VideoInfo(width=1280, height=720, fps=25, total_frames=750)


In [14]:
# Annotators
line_zone = sv.LineZone(START, END)
line_annotator = sv.LineZoneAnnotator(thickness=2, text_thickness=2, text_scale=1)

box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1, text_scale=0.5)

byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=video_info.fps)

def callback(frame: np.ndarray, index:int) -> np.ndarray:
    results = model(frame, verbose=False)[0]
        
    detections = sv.Detections.from_ultralytics(results)
    detections = detections[np.isin(detections.class_id, classes)]
    detections = byte_tracker.update_with_detections(detections)
    
    labels = [
        f'#{tracker_id} {model.model.names[class_id]} {confidence:.2f}'
        for _, _, confidence, class_id, tracker_id in detections
    ]
    
    annotated_frame = box_annotator.annotate(
        scene=frame.copy(),
        detections=detections,
        labels=labels
    )
        
    line_zone.trigger(detections)
        
    return line_annotator.annotate(annotated_frame, line_counter=line_zone)

# Processar cafa frame, visualizar e montar o vídeo
with sv.VideoSink(target_path=OUTPUT, video_info=video_info) as sink:
    for frame, index in enumerate(sv.get_video_frames_generator(source_path=VIDEO)):
        annotated_frame = callback(frame, index)
        sink.write_frame(frame=annotated_frame)
        
        # Mostra o vídeo
        cv2.imshow('frame', annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
cv2.destroyAllWindows()

# Contagem dos carros
carros = line_zone.in_count + line_zone.out_count
print(f'Carros: {carros}')

2023-12-03 14:55:32.974102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 14:55:32.974287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 14:55:33.041154: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[ WARN:0@32.044] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@32.044] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


ConnectionError: 1/1: 0... Failed to open 0